# 1. Introduction

This notebook connects to the Fresh Produce Database on SQL, cleans the tables and save the cleaned tables in the database.

!pip install pyodbc 

In [ ]:
#servername: DESKTOP-TTAV4PQ\SQLEXPRESS

In [2]:
#!pip install sqlalchemy
#!conda install pymssql
#apt-get -qq install python-pymssql python3-pymssql

SyntaxError: invalid syntax (<ipython-input-2-3860191b3a65>, line 3)

In [3]:
Importing modules
import pandas as pd
import numpy as np
import pyodbc

## Local Connection SQL

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-TTAV4PQ\SQLEXPRESS;'
                      'Database=Fresh_Produce_Market_Data;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

cpt_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Capetown_Fresh_produce_market',conn)
dbn_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Durban_Fresh_produce_market',conn)
jhb_comm_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_commodity_raw',conn)
jhb_con_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_container_raw',conn)
jhb_comb_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_product_combination_raw',conn)
woolworths_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.woolworths_Prices',conn)
pnp_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.PickNPay_Prices',conn)
shoprite_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Shoprite_Prices',conn)
#print(sql_query)
#print(type(sql_query))

## Remote Connection (RDS)

In [4]:
# creating a connection to the remote database (RDS)
cnxn = pyodbc.connect(
    DRIVER='{SQL Server}',
    SERVER='incubate-inventory-management.cnzbp4ndrpos.eu-west-1.rds.amazonaws.com,1433',
    DATABASE='Fresh_Produce_Market_Data',
    uid='admin',
    pwd='fXToh9AXt30Yi7dGdlI2') 

In [5]:
#Creating the pandas dataframe to store the data imported from RDS instance:
cpt_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Capetown_Fresh_produce_market',cnxn)
dbn_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Durban_Fresh_produce_market',cnxn)
jhb_comm_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_commodity_raw',cnxn)
jhb_con_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_container_raw',cnxn)
jhb_comb_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Joburg_Fresh_produce_product_combination_raw',cnxn)
woolworths_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.woolworths_Prices',cnxn)
pnp_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.PickNPay_Prices',cnxn)
shoprite_df = pd.read_sql_query('SELECT * FROM Fresh_Produce_Market_Data.dbo.Shoprite_Prices',cnxn)

In [6]:
# Exploring the data
cpt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3731 entries, 0 to 3730
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   item             3731 non-null   object
 1   description      3731 non-null   object
 2   grade            3731 non-null   object
 3   container        3731 non-null   object
 4   container_count  3731 non-null   int64 
 5   mass             3731 non-null   object
 6   low_price        3731 non-null   object
 7   high_price       3731 non-null   object
 8   avg_price        3731 non-null   object
 9   date_updated     3731 non-null   object
dtypes: int64(1), object(9)
memory usage: 291.6+ KB


In [7]:
#jhb_comm_df.head()
#jhb_con_df.head()
jhb_comb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14634 entries, 0 to 14633
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   rowid                 14634 non-null  int64  
 1   date                  14634 non-null  object 
 2   commodity             14634 non-null  object 
 3   container             14634 non-null  object 
 4   unit_mass             14634 non-null  int64  
 5   product_combination   14634 non-null  object 
 6   total_value_sold      14634 non-null  object 
 7   total_qty_sold        14634 non-null  int64  
 8   total_kg_sold         14634 non-null  float64
 9   average               14634 non-null  object 
 10  highest_price         14634 non-null  object 
 11  ave_per_kg            14634 non-null  object 
 12  highest_price_per_kg  14634 non-null  object 
dtypes: float64(1), int64(3), object(9)
memory usage: 1.5+ MB


# Creating Functions to clean the data

## The function to clean Rand Value columns  and normalise them

In [8]:
def rand_value(df, column):
    """
    Takes a dataframe and a column with rand value to be cleaned as arguments, 
    returns a dataframe, with two new columns with rand value for one day and
    Month To Date rand value. The initial column that was added as argument is removed from the dataframe.
    """ 
    
    df["MTD_total_value_sold_(R)"]=np.nan
    df["total_value_sold_(R)"]=np.nan
    for i in range(len(df[column])):
        df["MTD_total_value_sold_(R)"][i]=round(float(df[column][i].split("R")[-1].replace(',','')),2)
        df["total_value_sold_(R)"][i]=round(float(df[column][i].split("R")[1].split("M")[0].replace(",","")),2)
    
    df1=df.copy()    
    df1.drop(column,axis=1, inplace=True)
    return df1
    

In [9]:
jhb_comm_df_cleaned=rand_value(jhb_comm_df,"total_value_sold")
jhb_con_df_cleaned=rand_value(jhb_con_df,"value_sold")

C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
jhb_comm_df_cleaned.head()
jhb_con_df_cleaned.head()

,rowid,date,commodity,container,qty_available,qty_sold,kg_sold,average_price_per_kg,MTD_total_value_sold_(R),total_value_sold_(R)
0,1,20 August 2020,AMADUMBE,20KG POCKET,2,0MTD: 97,"0MTD: 1,940",R0,39870.0,0.0
1,2,20 August 2020,APPLES,10 X 1KG ECONO PACK CARTON,17,1MTD: 547,"10MTD: 5,470",R10,41932.0,100.0
2,3,20 August 2020,APPLES,11KG JUMBLE CARTON,343,"17MTD: 3,170","187MTD: 34,870",R6.36,218914.0,1190.0
3,4,20 August 2020,APPLES,12 X 1KG ECONO PACK CARTON,3233,"432MTD: 17,353","5184MTD: 208,236",R7.86,1454572.0,40738.0
4,5,20 August 2020,APPLES,12.5KG M6 CARTON,12,0MTD: 286,"0MTD: 3,575",R0,16903.0,0.0


## The function to clean quantities and weight columns simultaneuosly

In [11]:
def sold_qty_kg(df,quatity_sold, weight_sold):
    df["Total_quatity_sold"]=np.nan
    df["MTD_Total_quatity_sold"]=np.nan
    df["Total_kg_sold"]=np.nan
    df["MTD_total_kg_sold"]=np.nan
    for i in range(len(df[quatity_sold])):
        df["Total_quatity_sold"][i]=int(df[quatity_sold][i].split("M")[0].replace(",",""))
        df["MTD_Total_quatity_sold"][i]=int(df[quatity_sold][i].split(":")[-1].replace(',',''))
        df["Total_kg_sold"][i]=float(df[weight_sold][i].split("M")[0].replace(",",""))
        df["MTD_total_kg_sold"][i]=float(df[weight_sold][i].split(":")[-1].replace(',',''))
    df1=df.copy()
    df1.drop([quatity_sold,weight_sold],axis=1, inplace=True)
    return df1

In [12]:
jhb_comm_df_cleaned= sold_qty_kg(jhb_comm_df_cleaned, "total_qty_sold","total_kg_sold")
jhb_con_df_cleaned= sold_qty_kg(jhb_con_df_cleaned, "qty_sold","kg_sold")

C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launc

In [13]:
jhb_comm_df_cleaned.head()
#jhb_con_df_cleaned.head()

,rowid,date,commodity,qty_available,MTD_total_value_sold_(R),total_value_sold_(R),Total_quatity_sold,MTD_Total_quatity_sold,Total_kg_sold,MTD_total_kg_sold
0,1,20 August 2020,AMADUMBE,2,39870.0,0.0,0.0,97.0,0.0,1940.0
1,2,20 August 2020,APPLES,91755,22664221.0,1205932.0,13799.0,261296.0,157462.0,3163863.0
2,3,20 August 2020,ARTICHOKES,1,53100.0,600.0,4.0,439.0,3.0,522.0
3,4,20 August 2020,ASPARAGUS,8,258975.0,34000.0,50.0,359.0,250.0,1795.0
4,5,20 August 2020,ATCHARA,207,1351.2,0.0,0.0,23.0,0.0,65.0


In [14]:
def to_numeric(df,column):
    for i in range(len(df[column])):
        df[column][i]=df[column][i].replace(",","")
    df[column]= pd.to_numeric(df[column])
    return  df

In [15]:
cpt_df["date_updated"]=pd.to_datetime(cpt_df["date_updated"])
cpt_df=to_numeric(cpt_df,"mass")
cpt_df=to_numeric(cpt_df, "low_price")
cpt_df=to_numeric(cpt_df,"high_price")
cpt_df=to_numeric(cpt_df,"avg_price")

C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
dbn_df["Date"]=pd.to_datetime(dbn_df["Date"])
dbn_df=to_numeric(dbn_df,"Weight_Kg")
dbn_df=to_numeric(dbn_df,"Low_Price")
dbn_df=to_numeric(dbn_df,"High_Price")
dbn_df=to_numeric(dbn_df,"Average_Price")
dbn_df=to_numeric(dbn_df,"Sales_Total")
dbn_df=to_numeric(dbn_df,"Total_Qty_Sold")
dbn_df=to_numeric(dbn_df,"Total_Kg_Sold")
dbn_df=to_numeric(dbn_df,"Stock_On_Hand")
dbn_df.head()

C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Commodities,Weight_Kg,Size_Grade,Container,Province,Low_Price,High_Price,Average_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,Date
0,NOVAS,10.0,2M,BI100,CAPE,0.0,0.0,0.00,0.0,0,0.0,52,2020-07-15
1,NOVAS,10.0,2M,BI100,EASTERN CAPE,65.0,65.0,65.00,14560.0,224,2240.0,0,2020-07-15
2,NOVAS,15.0,2M,BN150,CAPE,70.0,75.0,70.71,495.0,7,105.0,0,2020-07-15
3,NOVAS,10.0,2S,BI100,CAPE,30.0,30.0,30.00,180.0,6,60.0,70,2020-07-15
4,NOVAS,10.0,2S,BI100,EASTERN CAPE,50.0,50.0,50.00,21700.0,434,4340.0,0,2020-07-15


In [17]:
jhb_comb_df["date"]=pd.to_datetime(jhb_comb_df["date"])
jhb_comb_df=rand_value(jhb_comb_df, "total_value_sold")
jhb_comb_df=rand_value(jhb_comb_df, "average")
jhb_comb_df=rand_value(jhb_comb_df, "highest_price")
jhb_comb_df=rand_value(jhb_comb_df, "ave_per_kg")
jhb_comb_df=rand_value(jhb_comb_df, "highest_price_per_kg")

C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\27784\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
#jhb_comb_df.info()

In [19]:
woolworths_df=to_numeric(woolworths_df, "Price")
woolworths_df["Date"]=pd.to_datetime(woolworths_df["Date"])

pnp_df=to_numeric(pnp_df, "Price")
pnp_df["Date"]=pd.to_datetime(pnp_df["Date"])

shoprite_df=to_numeric(shoprite_df, "Price")
shoprite_df["Date"]=pd.to_datetime(shoprite_df["Date"])

In [30]:
# Packages need
from sqlalchemy import create_engine

import urllib

params = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=incubate-inventory-management.cnzbp4ndrpos.eu-west-1.rds.amazonaws.com,1433;DATABASE=Fresh_Produce_Market_Data;UID=admin;pwd=fXToh9AXt30Yi7dGdlI2;trusted_connection=no")
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)





Username = 'admin'
Password = 'fXToh9AXt30Yi7dGdlI2'
Host = 'incubate-inventory-management.cnzbp4ndrpos.eu-west-1.rds.amazonaws.com'
Port = '1433'
Database = 'Fresh_Produce_Market_Data'

conn = create_engine(
    'mssql+pymssql://' +
    Username + ':' + Password + '@' + Host + ':' + Port + '/' + Database
)

In [37]:
#jhb_comm_df_cleaned.to_sql('Joburg_Fresh_produce_commodity_cleaned', con=engine)
#jhb_con_df_cleaned.to_sql('Joburg_Fresh_produce_container_cleaned', con=engine)
#jhb_comb_df.to_sql('Joburg_Fresh_produce_combined_cleaned', con=engine)
cpt_df.to_sql('Capetown_Fresh_produce_market_cleaned', con=engine)
dbn_df.to_sql('Durban_Fresh_produce_market_cleaned', con=engine)
woolworths_df.to_sql('Woolworths_Prices_cleaned', con=engine)
pnp_df.to_sql('PickNPay_Prices_cleaned', con=engine)
shoprite_df.to_sql('Shoprite_Prices_cleaned', con=engine)

DBAPIError: (pyodbc.Error) ('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionRead (recv()). (10054) (SQLExecDirectW); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]General network error. Check your network documentation. (11)')
[SQL: INSERT INTO [Durban_Fresh_produce_market_cleaned] ([index], [Commodities], [Weight_Kg], [Size_Grade], [Container], [Province], [Low_Price], [High_Price], [Average_Price], [Sales_Total], [Total_Qty_Sold], [Total_Kg_Sold], [Stock_On_Hand], [Date]) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: ((0, 'NOVAS', 10.0, '2M', 'BI100', 'CAPE', 0.0, 0.0, 0.0, 0.0, 0, 0.0, 52, datetime.datetime(2020, 7, 15, 0, 0)), (1, 'NOVAS', 10.0, '2M', 'BI100', 'EASTERN CAPE', 65.0, 65.0, 65.0, 14560.0, 224, 2240.0, 0, datetime.datetime(2020, 7, 15, 0, 0)), (2, 'NOVAS', 15.0, '2M', 'BN150', 'CAPE', 70.0, 75.0, 70.71, 495.0, 7, 105.0, 0, datetime.datetime(2020, 7, 15, 0, 0)), (3, 'NOVAS', 10.0, '2S', 'BI100', 'CAPE', 30.0, 30.0, 30.0,

In [ ]:
#engine.execute("SELECT * FROM Joburg_Fresh_produce_commodity_cleaned").fetchall()

In [ ]:
# Check that temp table is in your MS SQL Database
conn = pyodbc.connect(driver='{SQL Server}',
                      host='DESKTOP-TTAV4PQ\SQLEXPRESS',
                      database='Fresh_Produce_Market_Data',
                      trusted_connection='true',
                      user='sa')

In [ ]:
cursor.execute('SELECT * from Joburg_Fresh_produce_container_cleaned')

In [ ]:
#cursor.fetchall()

## Dropping a table

In [ ]:
sql = 'Drop TABLE Table_testing;'
conn.execute(sql)